# Performance Optimization with JSON Documents
Updated: 2019-10-03

## Optimizing Access to JSON Documents

An earlier notebook covered some of the differences between storing data in JSON format versus BSON format. From an `INSERT` perspective, if your incoming data is not already in BSON format, then it is more expensive to store data in BSON format, as shown in the following graph, since you will need to invoke the `JSON_TO_BSON` conversion function. 

The CUSTOMER document data set was used which includes 20,000 customer documents in JSON format with details on individual customers including an array of product purchases. The JSON column is defined as VARCHAR(2000), while the BSON column is defined as VARBINARY(2000) to avoid the additional overhead of dealing with BLOB objects.

### Load the Customer file into a table
You must run the first command to get the working directory for the IMPORT command.

In [ ]:
import os 
fname = os.getcwd() + "/customers.js"
print("Input file: " + fname)

### Load Db2 Extensions
The Db2 Jupyter extensions need to be loaded in order to run any of the examples in this notebook. In addition, a `CONNECT` command needs to be issued to connect to the local Db2 database. The default `SAMPLE` database is assumed to exist on the local system. If not, you need to modify the `CONNECT` command to use the appropriate userid, database, and host parameters.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

Next we create the table that will contain the customer data.

In [ ]:
%%sql -quiet 
DROP TABLE JSON_RAW_DATA;
CREATE TABLE JSON_RAW_DATA 
  (
  CUSTOMER VARCHAR(2000)
  );

The following code will load the data into the table.

In [ ]:
import io
import json
print("Starting Load")
start_time = time.time()
%sql autocommit off
x = %sql prepare INSERT INTO JSON_RAW_DATA VALUES (?)
if (x != False):
    i = 0
    with open(fname,"r") as records:
        for record in records:
            i += 1
            rc = %sql execute :x using record@char
            if (rc == False): break
            if ((i % 5000) == 0): 
                print(str(i)+" rows read.")
                %sql commit hold
                
    %sql commit work  
%sql autocommit on
end_time = time.time()
print('Total load time for {:d} records is {:.2f} seconds'.format(i,end_time-start_time))

In this step we will create three tables to hold the data: one using a character format, one using a BLOB format (inlined), and the third using a binary format.

In [ ]:
%%sql -q
DROP TABLE JSON_CHAR;
CREATE TABLE JSON_CHAR 
  (
    CUSTOMER VARCHAR(2000)
  );
    
DROP TABLE JSON_BINARY;
CREATE TABLE JSON_BINARY 
  (
    CUSTOMER VARBINARY(2000)
  );
    
DROP TABLE JSON_BLOB;
CREATE TABLE JSON_BLOB 
  (
    CUSTOMER BLOB(2000) INLINE LENGTH 2000
  );

The data from the base table will be inserted into these two tables using `INSERT INTO SELECT FROM` syntax. The performance of inserting data into each table is compared after the `INSERT` completes.

In [ ]:
%sql -q INSERT INTO JSON_CHAR SELECT * FROM JSON_RAW_DATA;
char_load = sqlelapsed
%sql -q INSERT INTO JSON_BINARY SELECT JSON_TO_BSON(CUSTOMER) FROM JSON_RAW_DATA;
binary_load = sqlelapsed
%sql -q INSERT INTO JSON_BLOB SELECT JSON_TO_BSON(CUSTOMER) FROM JSON_RAW_DATA;
blob_load = sqlelapsed
%sql -bar values ('CHAR',:char_load),('BINARY',:binary_load),('BLOB',:blob_load)

Converting the data to BSON does add overhead to the `INSERT` time. You may also note that the BLOB insert time is longer because Db2 must add control information to each object. If you are only storing and retrieving entire JSON documents then the conversion to BSON may not be unnecessary. 

## Searching and Retrieving JSON Documents
Db2 uses the BSON format internally for the processing done by the JSON access functions. The BSON format has the advantage of having already parsed the document into key-value pairs as well as having a tree structure available for easy traversal. JSON documents need to be converted internally to BSON to allow the Db2 functions to be able traverse them. Any data stored in JSON format that is accessed by these functions is first implicitly converted to BSON format and any result returned is converted back to JSON format (if this is requested). This overhead occurs for each unique access to the JSON data and can significantly impact the performance of a query.

This means that there are two areas where this implicit overhead from JSON to BSON can impact query performance when accessing a JSON document:
* How many values do you need to materialize as part of the SELECT column list
* How many values do you need to reference in the SQL predicates

We ran a number of sample tests to explore the performance impacts of the different choices (see the previous section). In the graphs that follow, 3 bars are shown with the labels JSON, Binary, and BLOB. They represent the following:

* JSON – Data stored as JSON in a VARCHAR column
* Binary – Data stored as BSON in a VARBINARY column 
* BLOB – Data stored as BSON in a BLOB (inlined) column

The Db2 JSON functions need to traverse a document for both display and predicate purposes. If JSON documents are identified by predicates on non-JSON columns, then storing the fields in JSON or BSON format makes little difference from the perspective of predicate processing. If the SQL requires columns or predicates based on the JSON data itself, then additional overhead is required to evaluate each predicate for JSON formatted documents. Finally, the actual retrieval of the target value will also incur conversion overhead (if needed).
```sql
SELECT COUNT(*) FROM JSON_TABLE
  WHERE
  JSON_VALUE(CUSTOMER, '$.contact.state' RETURNING CHAR(2)) = 'OH'
```
The above statement was repeated as many times as possible in a 10 second interval and the execution count (throughput) is shown on the chart below (higher is better!).

In [ ]:
%sql option runtime 10
count_json = %sql -t \
  SELECT COUNT(*) FROM JSON_CHAR WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
count_blob = %sql -t \
  SELECT COUNT(*) FROM JSON_BLOB WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
count_binary = %sql -t \
  SELECT COUNT(*) FROM JSON_BINARY WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
%sql -bar values ('CHAR',:count_json),('BINARY',:count_binary),('BLOB',:count_blob)
%sql option runtime 1


The graph highlights the performance benefits of storing the data as JSON, BSON in a BLOB, or BSON in a VARBINARY field. The performance between VARBINARY and BLOB fields may vary between runs, but generally VARBINARY has the additional benefit of faster retrieval speed because it doesn't need to deal with large object pointers and can reside directly on a buffered data page (which inlined LOB fields can also do but only for the portion that fits on the page). Note that VARBINARY is limited to approximately 32K documents (the maximum Db2 page size) so if your documents are larger than that you will need to use LOB storage.

If you examine the cost of retrieving columns in a SELECT list, the performance ratio is almost the same.

In [ ]:
%sql option runtime 10
count_json = %sql -t \
  SELECT JSON_VALUE(CUSTOMER, '$.contact.city' RETURNING CHAR(30)) \
    FROM JSON_CHAR WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
count_blob = %sql -t \
  SELECT JSON_VALUE(CUSTOMER, '$.contact.city' RETURNING CHAR(30)) \
    FROM JSON_BLOB WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
count_binary = %sql -t \
  SELECT JSON_VALUE(CUSTOMER, '$.contact.city' RETURNING CHAR(30)) \
    FROM JSON_BINARY WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
%sql -bar values ('CHAR',:count_json),('BINARY',:count_binary),('BLOB',:count_blob)
%sql option runtime 1

The decision to use BSON versus JSON as the storage format comes down to whether or not the application needs to regularly search for fields within a JSON document. If the majority of the JSON access is to store and retrieve entire documents, then the overhead of BSON conversion is unnecessary. However, if the access pattern to the JSON document is unknown, then it may be worthwhile to convert the documents to BSON for faster retrieval. The other option is to use indexes which is discussed in the next section.

## Indexing JSON Documents
If your application is always scanning documents for specific values using SQL predicates, then it may be worth placing indexes on the target fields. Db2 supports computed indexes (aka index on expression or expression-based index), which allows for the use of functions like `JSON_VALUE` to be used as part of the index definition. For instance, searching for a customer number will result in a scan against the table if no indexes are defined:
```sql
SELECT COUNT(*) 
  FROM CUSTOMERS 
  WHERE JSON_VALUE(DETAILS, '$.customerid' RETURNING INT) = 100000
```

To create an index on the customerid field, we use the `JSON_VALUE` function to extract the value from the JSON field.
```sql
CREATE INDEX IX_CUSTOMERID ON CUSTOMERS
  (JSON_VALUE(DETAILS,'$.customerid' RETURNING INT));
```

One consideration when creating indexes on JSON documents is that the `JSON_VALUE` function must include a  `RETURNING` clause. The `CREATE INDEX` statement cannot determine the data type from the command and it will raise an error message when it attempts to create the index.

In [ ]:
%%sql
CREATE INDEX IX_CUSTOMERID ON JSON_CHAR
  (JSON_VALUE(CUSTOMER,'$.customerid'))

Each of the three tables will have an index placed on the `contact.state` key so that we can see what the performance difference will be on our previous queries.

In [ ]:
%%sql
CREATE INDEX IX_STATE_CHAR ON JSON_CHAR
  (JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)));
CREATE INDEX IX_STATE_BINARY ON JSON_BINARY
  (JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2))); 
CREATE INDEX IX_STATE_BLOB ON JSON_BLOB
  (JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)));   

Now we can run our queries again to see what the results will be.

In [ ]:
%sql option runtime 10
count_json = %sql -t \
  SELECT COUNT(*) FROM JSON_CHAR WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
count_blob = %sql -t \
  SELECT COUNT(*) FROM JSON_BLOB WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
count_binary = %sql -t \
  SELECT COUNT(*) FROM JSON_BINARY WHERE JSON_VALUE(CUSTOMER,'$.contact.state' RETURNING CHAR(2)) = 'OH'
%sql -bar values ('CHAR',:count_json),('BINARY',:count_binary),('BLOB',:count_blob)
%sql option runtime 1

In this scenario, where we have an SQL statement selecting which records to read based on predicates on JSON values, we can see that running the  statement across all three storage options results in uniform performance. This is because we do not actually access the JSON document to evaluate the predicate at execution time due to the use of the index in the access plan.

## Summary
As always, the time you need to spend on performance considerations will depend on the way that your JSON documents will be accessed by your application(s) and the performance requirements of the application. And the decisions you make will have to balance the benefits and cost of each possible solution. 
If you are going to do a lot of individual key access on your JSON documents or want to maximize your performance, then converting any incoming JSON data to BSON format as the data is stored will improve performance at time of access. As well, the process of normal query performance tuning may indicate that indexes on key JSON predicates will help performance considerably.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]